In [18]:
import pandas as pd
import re
# import json
# import pathlib
import datetime
pd.set_option('display.max_columns', 50)

In [1]:
from race import Race
from races import Races

r = Race()
tpls = r.races()
code = tpls[0][1]
rs = Races(code)
races = rs.oneday()
print(races)

[('1R 2歳未勝利  芝1800m サラ系２歳 未勝利 (混) 馬齢 8頭',    枠  馬  番         馬名  性齢    斤量    騎手       厩舎  馬体重  (増減)
0  1     1   トップスティール  牡2  54.0    丹内   美浦  深山  502  (-6)
1  2     2    ポイズンアロウ  牡2  54.0   横山武  美浦  藤沢和  528  (+2)
2  3     3     リュカイオス  牡2  54.0    菱田  栗東  杉山佳  428  (+2)
3  4     4    ヒルノロワール  牡2  54.0    池添   栗東  北出  486  (-2)
4  5     5     マイムマイム  牝2  51.0    小沢   美浦  小島  410  (-6)
5  6     6  ジョーカーブラウン  牡2  52.0   秋山稔   美浦  松山  510  (-6)
6  7     7     アンビション  牡2  54.0  ルメール   美浦  手塚  542  (+4)
7  8     8    オーシンプラス  牡2  54.0   古川吉    栗東  宮  462  (+8)), ('2R 2歳未勝利  ダ1700m サラ系２歳 未勝利   馬齢 11頭',     枠  馬  番        馬名  性齢    斤量    騎手       厩舎   馬体重  (増減)
0   1     1    フォルトゥナ  牡2  54.0    菱田   美浦  中野   462  (-2)
1   2     2   グレートバリア  牡2  52.0   秋山稔   美浦  相沢  474  (-14)
2   3     3  コスモガラクシア  牡2  54.0    丹内   美浦  金成    468  (0)
3   4     4   デルマドワーフ  牡2  54.0    菊沢   美浦  菊沢   450  (-2)
4   5     5       メルト  牡2  54.0    的場   美浦  的場   486  (+4)
5   6     6  コースタルテラス  牡2  54.0  ルメール   

In [88]:
url = "https://race.netkeiba.com/race/shutuba.html?race_id=202101020301"
soup = rs.get_soup(url)

In [89]:
soup

<!DOCTYPE html>
<html>
<head>
<meta charset="utf-8"/>
<!-- block=common__meta_tag_each_race (cp) -->
<meta content="ja" http-equiv="content-language"/>
<meta content="IE=edge" http-equiv="X-UA-Compatible"/>
<meta content="width=1000" name="viewport"/>
<meta content="telephone=no" name="format-detection"/>
<meta content="2021年8月21日 札幌1R ２歳未勝利の出馬表です。JRA開催レースの出馬表や最新オッズ、レース結果速報、払戻情報をはじめ、競馬予想やデータ分析など予想に役立つ情報も満載です。" name="description"/>
<meta content="競馬,keiba,出馬表,オッズ,予想,レース結果,払戻し,結果速報,競馬予想,ネット競馬,netkeiba" name="keywords"/>
<meta content="https://www.netkeiba.com/style/netkeiba.ja/image/netkeiba.png" name="thumbnail"/>
<!-- ogp用 -->
<meta content="netkeiba.com" property="og:site_name"/>
<meta content="article" property="og:type"/>
<meta content="２歳未勝利 出馬表 | 2021年8月21日 札幌1R レース情報(JRA) - netkeiba.com" property="og:title"/>
<meta content="https://race.netkeiba.com/race/shutuba.html?race_id=202101020301" property="og:url"/>
<meta content="2021年8月21日 札幌1R ２歳未勝利の出馬表です。JRA開催レースの出馬表や最新オッズ、レース結果速報、払戻

In [90]:
r = soup.select_one("div.RaceList_Item01 span.RaceNum").text
title = soup.select_one("div.RaceName").text.split()[0]
course = soup.select("div.RaceData01 span")[0].text
print(r, title, course)

1R 2歳未勝利  芝1800m


In [99]:
dfs = rs.get_dfs(soup)
df = dfs[0].droplevel(0, axis=1)
srs = []
for idx, sr in df.iterrows():
    srs.append(sr.drop(index=["印", "人気", "登録", "メモ"])[:-1])
race_df = pd.DataFrame(srs)
race_df

,枠,馬 番,馬名,性齢,斤量,騎手,厩舎,馬体重 (増減)
0,1,1,トップスティール,牡2,54.0,丹内,美浦 深山,502 (-6)
1,2,2,ポイズンアロウ,牡2,54.0,横山武,美浦 藤沢和,528 (+2)
2,3,3,リュカイオス,牡2,54.0,菱田,栗東 杉山佳,428 (+2)
3,4,4,ヒルノロワール,牡2,54.0,池添,栗東 北出,486 (-2)
4,5,5,マイムマイム,牝2,51.0,小沢,美浦 小島,410 (-6)
5,6,6,ジョーカーブラウン,牡2,52.0,秋山稔,美浦 松山,510 (-6)
6,7,7,アンビション,牡2,54.0,ルメール,美浦 手塚,542 (+4)
7,8,8,オーシンプラス,牡2,54.0,古川吉,栗東 宮,462 (+8)


In [97]:
# soup.select("div[class^=span3]")
ubans = [tag for tag in soup.select("td[class^=Umaban]")]
names = [tag for tag in soup.select("span.HorseName a")]
ages = [tag.text for tag in soup.select("td[class^=Barei]")]
jockeys = [tag.text.strip("\n") for tag in soup.select("td.Jockey")]
print(ages, jockeys)

['牡2', '牡2', '牡2', '牡2', '牝2', '牡2', '牡2', '牡2'] ['丹内', '横山武', '菱田', '池添', '小沢', '秋山稔', 'ルメール', '古川吉']


In [83]:
links = [(u.text, n.text, n.get("href")) for u, n in zip(ubans, names)]
links

[('1', 'トップスティール', 'https://db.netkeiba.com/horse/2019106387'),
 ('2', 'ポイズンアロウ', 'https://db.netkeiba.com/horse/2019110073'),
 ('3', 'リュカイオス', 'https://db.netkeiba.com/horse/2019102515'),
 ('4', 'ヒルノロワール', 'https://db.netkeiba.com/horse/2019106038'),
 ('5', 'マイムマイム', 'https://db.netkeiba.com/horse/2019106229'),
 ('6', 'ジョーカーブラウン', 'https://db.netkeiba.com/horse/2019103488'),
 ('7', 'アンビション', 'https://db.netkeiba.com/horse/2019105335'),
 ('8', 'オーシンプラス', 'https://db.netkeiba.com/horse/2019103276')]

In [84]:
rs = Race()

data = []
for no, name, link in links:
    soup = rs.get_soup(link)
    dfs = rs.get_dfs(soup)
    if len(dfs) > 3:
        df = dfs[3]
        lst = []
        for idx, row in df.iterrows():
            date = row["日付"]
            hld = row["開催"]
            r = row["R"]
            race = row["レース名"]
            wei = row["斤  量"]
            grd, dst = row["距離"][0], int(row["距離"][1:])
            cnd = row["馬  場"]
            time_str = row["タイム"]
            dt = datetime.datetime.strptime(time_str, "%M:%S.%f")
            td = datetime.timedelta(minutes=dt.minute, seconds=dt.second, microseconds=dt.microsecond)
            sec = td.total_seconds()
            lst.append((no, name, idx, date, hld, r, race, wei, grd, dst, sec))
        data.append(lst)

In [85]:
data

[[('1', 'トップスティール', 0, '2021/08/21', '2札幌3', 1, '2歳未勝利', 54, '芝', 1800, 109.9),
  ('1', 'トップスティール', 1, '2021/08/14', '2札幌1', 5, '2歳新馬', 54, '芝', 1500, 90.6)],
 [('2', 'ポイズンアロウ', 0, '2021/08/21', '2札幌3', 1, '2歳未勝利', 54, '芝', 1800, 110.2),
  ('2', 'ポイズンアロウ', 1, '2021/08/07', '1函館11', 1, '2歳未勝利', 54, '芝', 1800, 110.6),
  ('2', 'ポイズンアロウ', 2, '2021/07/24', '1函館7', 5, '2歳新馬', 54, 'ダ', 1700, 109.9)],
 [('3', 'リュカイオス', 0, '2021/08/21', '2札幌3', 1, '2歳未勝利', 54, '芝', 1800, 114.6),
  ('3', 'リュカイオス', 1, '2021/08/08', '1函館12', 5, '2歳新馬', 54, '芝', 1800, 114.0)],
 [('4', 'ヒルノロワール', 0, '2021/08/21', '2札幌3', 1, '2歳未勝利', 54, '芝', 1800, 110.5),
  ('4', 'ヒルノロワール', 1, '2021/08/07', '1函館11', 1, '2歳未勝利', 54, '芝', 1800, 110.6),
  ('4', 'ヒルノロワール', 2, '2021/07/18', '1函館6', 5, '2歳新馬', 54, '芝', 1800, 111.7)],
 [('5', 'マイムマイム', 0, '2021/08/21', '2札幌3', 1, '2歳未勝利', 51, '芝', 1800, 110.5),
  ('5', 'マイムマイム', 1, '2021/07/11', '1函館4', 5, '2歳新馬', 51, '芝', 1800, 113.3)],
 [('6',
   'ジョーカーブラウン',
   0,
   '2021/08/21',
   '2

In [15]:
dfs[1]

,0,1
0,生年月日,2019年3月30日
1,調教師,深山雅史 (美浦)
2,馬主,井山登
3,生産者,千代田牧場
4,産地,新ひだか町
5,セリ取引価格,"1,705万円 (2021年 北海道トレーニングセール)"
6,獲得賞金,240万円 (中央)
7,通算成績,2戦0勝 [ 0-0-1-1 ]
8,主な勝鞍,NaN
9,近親馬,ナリノリリー 、 オオエレーワダン


In [20]:
df = dfs[3]
df

,日付,開催,天 気,R,レース名,映 像,頭 数,枠 番,馬 番,オ ッ ズ,人 気,着 順,騎手,斤 量,距離,馬 場,馬場 指数,タイム,着差,ﾀｲﾑ 指数,通過,ペース,上り,馬体重,厩舎 ｺﾒﾝﾄ,備考,勝ち馬 (2着馬),賞金
0,2021/08/21,2札幌3,晴,1,2歳未勝利,NaN,8,1,1,22.7,5,3,丹内祐次,54,芝1800,良,NaN,1:49.9,0.3,NaN,4-4-3-2,36.1-37.0,37.1,502(-6),NaN,NaN,アンビション,130.0
1,2021/08/14,2札幌1,晴,5,2歳新馬,NaN,9,2,2,9.8,3,4,丹内祐次,54,芝1500,良,**,1:30.6,1.5,**,7-7-7,30.3-35.4,35.7,508(0),NaN,NaN,モンゴリアンキング,110.0


In [52]:
lst = []
for idx, row in df.iterrows():
    date = row["日付"]
    hld = row["開催"]
    r = row["R"]
    race = row["レース名"]
    wei = row["斤  量"]
    grd, dst = row["距離"][0], int(row["距離"][1:])
    cnd = row["馬  場"]
    time_str = row["タイム"]
    dt = datetime.datetime.strptime(time_str, "%M:%S.%f")
    td = datetime.timedelta(minutes=dt.minute, seconds=dt.second, microseconds=dt.microsecond)
    sec = td.total_seconds()
    lst.append((idx, date, hld, r, race, wei, grd, dst, sec))
lst

[(0, '2021/08/21', '2札幌3', 1, '2歳未勝利', 54, '芝', 1800, 109.9),
 (1, '2021/08/14', '2札幌1', 5, '2歳新馬', 54, '芝', 1500, 90.6)]

In [86]:
ls = [1,2,3]
ls.extend([4,5])
ls

[1, 2, 3, 4, 5]

In [92]:
sr = pd.Series([1,2,3], index=["one", "two", "three"])
list(sr)

[1, 2, 3]

In [115]:
sr = pd.Series(dtype=np.float64)
sr

Series([], dtype: float64)